In [ ]:
pip install torch torchvision gensim

In [ ]:
!python -m spacy download el_core_news_sm

In [ ]:
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
spacy.load('el_core_news_sm')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nlp = spacy.load("el_core_news_sm")
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import numpy as np

import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.metrics
import seaborn as sns
import random
import sys

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Import data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv("/kaggle/input/ys19-2023-assignment-3/train_set.csv")
test_df = pd.read_csv("/kaggle/input/ys19-2023-assignment-3/test_set.csv")
valid_df = pd.read_csv("/kaggle/input/ys19-2023-assignment-3/valid_set.csv")

df['Label_Map'] = df['Sentiment'].map({
    'NEGATIVE' : 0, 'NEUTRAL' : 1, 'POSITIVE' : 2
})
valid_df['Label_Map'] = valid_df['Sentiment'].map({
    'NEGATIVE' : 0, 'NEUTRAL' : 1, 'POSITIVE' : 2
})

# Preprocessing functions

In [ ]:
def lowercase_without_accentds(text):
    chars = "ΆΈΊΌΎΉάέίόύήώ"
    lchars = "αειουηαειουηω"
    return text.translate(str.maketrans(chars,lchars))

def extract_greek_characters(input_string):
    greek_pattern = re.compile('[Α-Ωα-ω]+')
    greek_characters = greek_pattern.findall(input_string)
    result = ' '.join(greek_characters)
    return result

greek_stopwords = ["ο","ειναι","απο","https","co","λεπω","λεμε","ειπα","ειπες","εβαλα","αδιακοπα","αι","ακομα","ακομη","ακριβως","αληθεια","αληθινα","αλλα","αλλαχου","αλλες","αλλη","αλλην","αλλης","αλλιως","αλλιωτικα","αλλο","αλλοι","αλλοιως","αλλοιωτικα","αλλον","αλλος","αλλοτε","αλλου","αλλους","αλλων","αμα","αμεσα","αμεσως","αν","ανα","αναμεσα","αναμεταξυ","ανευ","αντι","αντιπερα","αντις","ανω","ανωτερω","αξαφνα","απ","απεναντι","απο","αποψε","αρα","αραγε","αργα","αργοτερο","αριστερα","αρκετα","αρχικα","ας","αυριο","αυτα","αυτες","αυτη","αυτην","αυτης","αυτο","αυτοι","αυτον","αυτος","αυτου","αυτους","αυτων","αφοτου","αφου","βεβαια","βεβαιοτατα","γι","για","γρηγορα","γυρω","δα","δε","δεινα","δεν","δεξια","δηθεν","δηλαδη","δι","δια","διαρκως","δικα","δικο","δικοι","δικος","δικου","δικους","διολου","διπλα","διχως","εαν","εαυτο","εαυτον","εαυτου","εαυτους","εαυτων","εγκαιρα","εγκαιρως","εγω","εδω","ειδεμη","ειθε","ειμαι","ειμαστε","ειναι","εις","εισαι","εισαστε","ειστε","ειτε","ειχα","ειχαμε","ειχαν","ειχατε","ειχε","ειχες","εκαστα","εκαστες","εκαστη","εκαστην","εκαστης","εκαστο","εκαστοι","εκαστον","εκαστος","εκαστου","εκαστους","εκαστων","εκει","εκεινα","εκεινες","εκεινη","εκεινην","εκεινης","εκεινο","εκεινοι","εκεινον","εκεινος","εκεινου","εκεινους","εκεινων","εκτος","εμας","εμεις","εμενα","εμπρος","εν","ενα","εναν","ενας","ενος","εντελως","εντος","εντωμεταξυ","ενω","εξ","εξαφνα","εξης","εξισου","εξω","επανω","επειδη","επειτα","επι","επισης","επομενως","εσας","εσεις","εσενα","εστω","εσυ","ετερα","ετεραι","ετερας","ετερες","ετερη","ετερης","ετερο","ετεροι","ετερον","ετερος","ετερου","ετερους","ετερων","ετουτα","ετουτες","ετουτη","ετουτην","ετουτης","ετουτο","ετουτοι","ετουτον","ετουτος","ετουτου","ετουτους","ετουτων","ετσι","ευγε","ευθυς","ευτυχως","εφεξης","εχει","εχεις","εχετε","εχθες","εχομε","εχουμε","εχουν","εχτες","εχω","εως","η","ηδη","ημασταν","ημαστε","ημουν","ησασταν","ησαστε","ησουν","ηταν","ητανε","ητοι","ηττον","θα","ι","ιδια","ιδιαν","ιδιας","ιδιες","ιδιο","ιδιοι","ιδιον","ιδιος","ιδιου","ιδιους","ιδιων","ιδιως","ιι","ιιι","ισαμε","ισια","ισως","καθε","καθεμια","καθεμιας","καθενα","καθενας","καθενος","καθετι","καθολου","καθως","και","κακα","κακως","καλα","καλως","καμια","καμιαν","καμιας","καμποσα","καμποσες","καμποση","καμποσην","καμποσης","καμποσο","καμποσοι","καμποσον","καμποσος","καμποσου","καμποσους","καμποσων","κανεις","κανεν","κανενα","κανεναν","κανενας","κανενος","καποια","καποιαν","καποιας","καποιες","καποιο","καποιοι","καποιον","καποιος","καποιου","καποιους","καποιων","καποτε","καπου","καπως","κατ","κατα","κατι","κατιτι","κατοπιν","κατω","κιολας","κλπ","κοντα","κτλ","κυριως","λιγακι","λιγο","λιγωτερο","λογω","λοιπα","λοιπον","μα","μαζι","μακαρι","μακρυα","μαλιστα","μαλλον","μας","με","μεθαυριο","μειον","μελει","μελλεται","μεμιας","μεν","μερικα","μερικες","μερικοι","μερικους","μερικων","μεσα","μετ","μετα","μεταξυ","μεχρι","μη","μηδε","μην","μηπως","μητε","μια","μιαν","μιας","μολις","μολονοτι","μοναχα","μονες","μονη","μονην","μονης","μονο","μονοι","μονομιας","μονος","μονου","μονους","μονων","μου","μπορει","μπορουν","μπραβο","μπρος","να","ναι","νωρις","ξανα","ξαφνικα","ο","οι","ολα","ολες","ολη","ολην","ολης","ολο","ολογυρα","ολοι","ολον","ολονεν","ολος","ολοτελα","ολου","ολους","ολων","ολως","ολωςδιολου","ομως","οποια","οποιαδηποτε","οποιαν","οποιανδηποτε","οποιας","οποιαςδηποτε","οποιδηποτε","οποιες","οποιεςδηποτε","οποιο","οποιοδηποτε","οποιοι","οποιον","οποιονδηποτε","οποιος","οποιοςδηποτε","οποιου","οποιουδηποτε","οποιους","οποιουςδηποτε","οποιων","οποιωνδηποτε","οποτε","οποτεδηποτε","οπου","οπουδηποτε","οπως","ορισμενα","ορισμενες","ορισμενων","ορισμενως","οσα","οσαδηποτε","οσες","οσεςδηποτε","οση","οσηδηποτε","οσην","οσηνδηποτε","οσης","οσηςδηποτε","οσο","οσοδηποτε","οσοι","οσοιδηποτε","οσον","οσονδηποτε","οσος","οσοςδηποτε","οσου","οσουδηποτε","οσους","οσουςδηποτε","οσων","οσωνδηποτε","οταν","οτι","οτιδηποτε","οτου","ου","ουδε","ουτε","οχι","παλι","παντοτε","παντου","παντως","παρα","περα","περι","περιπου","περισσοτερο","περσι","περυσι","πια","πιθανον","πιο","πισω","πλαι","πλεον","πλην","ποια","ποιαν","ποιας","ποιες","ποιο","ποιοι","ποιον","ποιος","ποιου","ποιους","ποιων","πολυ","ποσες","ποση","ποσην","ποσης","ποσοι","ποσος","ποσους","ποτε","που","πουθε","πουθενα","πρεπει","πριν","προ","προκειμενου","προκειται","προπερσι","προς","προτου","προχθες","προχτες","πρωτυτερα","πως","σαν","σας","σε","σεις","σημερα","σιγα","σου","στα","στη","στην","στης","στις","στο","στον","στου","στους","στων","συγχρονως","συν","συναμα","συνεπως","συνηθως","συχνα","συχνας","συχνες","συχνη","συχνην","συχνης","συχνο","συχνοι","συχνον","συχνος","συχνου","συχνου","συχνους","συχνων","συχνως","σχεδον","σωστα","τα","ταδε","ταυτα","ταυτες","ταυτη","ταυτην","ταυτης","ταυτο,ταυτον","ταυτος","ταυτου","ταυτων","ταχα","ταχατε","τελικα","τελικως","τες","τετοια","τετοιαν","τετοιας","τετοιες","τετοιο","τετοιοι","τετοιον","τετοιος","τετοιου","τετοιους","τετοιων","τη","την","της","τι","τιποτα","τιποτε","τις","το","τοι","τον","τος","τοσα","τοσες","τοση","τοσην","τοσης","τοσο","τοσοι","τοσον","τοσος","τοσου","τοσους","τοσων","τοτε","του","τουλαχιστο","τουλαχιστον","τους","τουτα","τουτες","τουτη","τουτην","τουτης","τουτο","τουτοι","τουτοις","τουτον","τουτος","τουτου","τουτους","τουτων","τυχον","των","τωρα","υπ","υπερ","υπο","υποψη","υποψιν","υστερα","φετος","χαμηλα","χθες","χτες","χωρις","χωριστα","ψηλα","ω","ωραια","ως","ωσαν","ωσοτου","ωσπου","ωστε","ωστοσο","ωχ","ο","η","το","τα","τη","δηλαδη","μεχρι","γιατι","εχω","στους","μια","ένας","μία","κάποιος","κάποια","κάποιο","κάποιοι","αυτος","αυτη","αυτο","αυτοι","αυτες","αυτα","στο","στη","στα","για","με","απο","προς","ειναι","εχει","εχουν","θα","δεν","πανω","κατω","μεσα","εξω","κατω","ως","πανω","κατω","πιο","εδω","εκει","πολυ","λιγο","τωρα","ακομα","ομως","επισης","παντα","ακομη","πιθανως","μονο","οχι","ναι","ευχαριστως","γενικα","ολοι","ολες","ολα","ποιος","ποια","ποιο","ποιοι","ποιες","τιποτα","κανεις","καμια","κανενα","κανενες","αυτος","αυτη","αυτο","αυτοι","αυτες","αυτα","απο","σε","υπο","μετα","πριν","επειτα","αντι","εναν","μιαν","κανεναν","καμιαν","κανενα","καμια","μιαν","ενα","οποιος","οποια","οποιο","οποιοι","οποιες","οποιαν","οποιον","ολος","ολη","ολα","ολους","ολες","ολων","καθενας","καθεμια","καθενα","καθενες","ακομα","ενω","επομενως","συνεπως","επιπλεον","παρολα αυτα","παρ ολα αυτα","επισης","και","αλλα","αλλα και","αν","εαν","αν και","αντι","αντι να","αντι το","αντι τα","αντι του","αντι τη","αντι των","αντι στο","αντι στη","αντι στα","αντι στου","αντι στην","αντι στις","αντι στον","μεσω","τους","μας","ηταν","εκ","φορα","πρωτη","ειχα","εμεις","εσεις","ηδη","απ","εγινε","ειχε","αλλα","ουτε","ενας","εσας","αυτοι","αυτο","νεα","οντως","θελετε","κανει","σ","μας","πρεπε","ε","μαλιστα","τους","ηθελε","παω","εβαλε","λεει","γ","ν","θες","ερχεται","διαρκεια","θελουν","ασε","χ","λες","ξερω","α","δω","ειδε","μπηκε","βαλει","μερες","εφοσον","ενα","δυο","τρια","γινει","εργο","μιλαω","μιλησε","ποσα","ωρες","πρωινες","πρωτα","θελει","βαζω","εβαζε","εναντι","μπορεις","βρισκει","δει","μπορω","γινε","κανουν"]
pattern = '#!$%^&*()_+={}\[\]:;"\'<>,.?/\\|`~-><]'

def process_text(text):
    
#     new_text = extract_greek_characters(text)
    doc = nlp(text)
    processed_words = []
    # Loop through each word
    for word in doc:
        if word.is_punct or any(char.isdigit() for char in word.text) or word.text in greek_stopwords:
            continue
        else:
            clean_word = word.lemma_
            clean_word = lowercase_without_accentds(clean_word)
            clean_word = clean_word.lower()
            if clean_word not in greek_stopwords:
                processed_words.append(clean_word)
    # Join the processed words back into a string
    processed_text = ' '.join(processed_words)
    return processed_text

# Data preprocessing

In [ ]:
import os
# os.remove('train.json')
# os.remove('valid.json')
text = df['Text']
valid_text = valid_df['Text']
test_text = test_df['Text']
print("Preproccesing training data")
df['processed_text'] = text.apply(process_text)
df.to_json('train.json', orient='records', lines=True)
print("Preproccesing valid data")
valid_df['processed_text'] = valid_text.apply(process_text)
valid_df.to_json('valid.json', orient='records', lines=True)
test_df['processed_text'] = test_text.apply(process_text)
test_df.to_json('test.json', orient='records', lines=True)
print("Vectorization")

# Get preprocessed data from file

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Write the DataFrame to a JSON file
df_read = pd.read_json('/kaggle/input/processed-dataset/train.json', lines=True)
train_preprocessed_text = df_read['processed_text']
valid_df_read = pd.read_json('/kaggle/input/processed-dataset/valid.json', lines=True)
valid_preprocessed_text = valid_df_read['processed_text']
test_df_read = pd.read_json('/kaggle/input/processed-dataset/test.json', lines=True)
test_preprocessed_text = test_df_read['processed_text']

print("TRAIN")
print(len(train_preprocessed_text))
print("VALID")
print(len(valid_preprocessed_text))
print("TEST")
print(len(test_preprocessed_text))

# Data tokenization

In [ ]:
import re
from nltk.tokenize import word_tokenize

def clean_and_tokenize(text):

    pattern = r'[!@#$%^&*()_+={}\[\]:;<>,.?/~`]+'
    
    cleaned_text = re.sub(pattern, '', text)

    tokens = cleaned_text.split()
    
    return tokens

# Tokenize model

In [ ]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

# Preprocess each sentence
df_read['Tokenized_text'] = train_preprocessed_text.apply(clean_and_tokenize)
valid_df_read['Tokenized_text'] = valid_preprocessed_text.apply(clean_and_tokenize)
test_df_read['Tokenized_text'] = test_preprocessed_text.apply(clean_and_tokenize)

# Visualize model

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

# Assuming df_read and valid_df_read are your dataframes with 'Tokenized_text' column

# Combine the tokenized text from both dataframes
all_tokenized_text = pd.concat([df_read['Tokenized_text'], valid_df_read['Tokenized_text'],test_df_read['Tokenized_text']])

# Define the vector_size (you need to set it to the appropriate value)
vector_size = 100

# Visualized words
visualized_words = ['μητσοτακης', 'τσιπρας', 'νδ', 'συριζα', 'κουλη', 'κκε', 'πολιτης', 'τσιπρα', 'βουλευτης', 'κυριακος',
                   'προθυπουργος' , 'πασοκ' , 'πολιτικος' , 'δημοκρατια' , 'νεος' , 'βελοπουλος']

def visualize_embeddings(embeddings, words, axs, title):
    tsne = TSNE(init = "pca",n_components=2,random_state=42, perplexity=len(words) - 1)
    embedding_vectors = np.array([embeddings[word] for word in words])
    two_d_embeddings = tsne.fit_transform(embedding_vectors)

    for i, word in enumerate(words):
        x, y = two_d_embeddings[i, :]
        axs.scatter(x, y)
        axs.annotate(word, (x, y), xytext=(5, 2), textcoords="offset points", ha="right", va="bottom")

    axs.set_title(title)

# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# # Iterate over the subplots
count = 2
for i in range(2):
    for j in range(3):
        axs_ = axs[i, j]

        # Train Word2Vec model with CBOW algorithm (sg=0)
        w2v_model = Word2Vec(sentences=all_tokenized_text, vector_size=vector_size, window = count, min_count=1, sg=0)
        
        # Visualize embeddings
        visualize_embeddings(w2v_model.wv, visualized_words, axs_, f'CBOW, Window Size: {count}')
        count += 3
# Show the plots
plt.show()

# Create new subplots for the second set of visualizations
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
count = 2
for i in range(2):
    for j in range(3):
        axs_ = axs[i, j]

        # Train Word2Vec model with Skip-gram algorithm (sg=1)
        w2v_model = Word2Vec(sentences=all_tokenized_text, vector_size=vector_size, window=count, min_count=1, sg=1)

        # Visualize embeddings
        visualize_embeddings(w2v_model.wv, visualized_words, axs_, f'Skip-gram, Window Size: {count}')
        count += 3

# Show the plots
plt.show()

# Train word2vec

In [ ]:
vector_size = 100
w2v_model = Word2Vec(sentences=pd.concat([df_read['Tokenized_text'], valid_df_read['Tokenized_text'],test_df_read['Tokenized_text']]), vector_size=vector_size, window=8, min_count=1, sg=0)
w2v_model.save("greek_word2vec.model")

# Create training torches

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from gensim.models import Word2Vec
from statistics import mean

# Load the Word2Vec model
w2v_model = Word2Vec.load("greek_word2vec.model")

# Function to get word embeddings for a tokenized tweet
def get_word_embeddings(tokenized_tweet, model):
    embeddings = []
    if not tokenized_tweet:
        tokenized_tweet = ["Ολυμπιακός"] # word without sence
    for word in tokenized_tweet:        
        if word in model.wv:
            embeddings.append(model.wv[word])
        else:
            embeddings.append(np.zeros(model.vector_size, dtype=np.float32))
    return embeddings

# Function to get mean embeddings for a list of tweets
def get_embeddings_by_mean(tweets):
    return np.mean(np.stack(tweets, axis=0), axis=0)

def padding_and_torching(l):
    max_size = 0
    list_of_lengths = []
    for tweet in l:
        list_of_lengths.append(len(tweet))
        if(len(tweet) > max_size):
            max_size = len(tweet)
    array_of_max = np.zeros(100, dtype=np.float64)
    
    padded_tweets = [tweet + [array_of_max] * (max_size - len(tweet)) for tweet in l]
    t = torch.tensor(padded_tweets, device = device)
    t1 = torch.tensor(list_of_lengths)
    return t,t1

word_embeddings = [get_word_embeddings(tokenized_tweet, w2v_model) for tokenized_tweet in df_read['Tokenized_text']]
word_embeddings_valid = [get_word_embeddings(tokenized_tweet, w2v_model) for tokenized_tweet in valid_df_read['Tokenized_text']]
word_embeddings_test = [get_word_embeddings(tokenized_tweet, w2v_model) for tokenized_tweet in test_df_read['Tokenized_text']]

X_train,X_train_length = padding_and_torching(word_embeddings)
X_valid,X_valid_length = padding_and_torching(word_embeddings_valid)
X_test,X_test_lenth = padding_and_torching(word_embeddings_test)


# Convert to tensors
# X_train = torch.tensor([get_embeddings_by_mean(tweet) for tweet in word_embeddings])
# X_valid = torch.tensor([get_embeddings_by_mean(tweet) for tweet in word_embeddings_valid])
# X_test = torch.tensor([get_embeddings_by_mean(tweet) for tweet in word_embeddings_test])

# # Convert labels to tensors
Y_train = torch.tensor(df_read['Label_Map'],device = device)
Y_valid = torch.tensor(valid_df_read['Label_Map'], device = device)


X_train = X_train.to(device)
X_valid = X_valid.to(device)
X_test = X_test.to(device)

Y_train = Y_train.to(device)
Y_valid = Y_valid.to(device)
print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)
print(X_test.shape)
print(X_train_length.shape)
print(X_valid_length.shape)

# Roc curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

def plot_roc_curve(y_true, probabilities):
    # Assuming probabilities is a list of numpy arrays
#     probabilities = np.vstack(probabilities)
    probabilities = np.vstack([p.detach().cpu().numpy() for p in probabilities])

    # Binarize the labels
    y_true_bin = label_binarize(y_true, classes=[0, 1, 2])

    num_classes = probabilities.shape[1]

    plt.figure(figsize=(8, 8))  # Adjust the figure size as needed

    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], probabilities[:, i])
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f'Class {i} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

# Rnn class

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    """
      Wrapper module for RNN, LSTM, GRU cells using pytorch api. The output is fed to a FFN for classification.
    """
    def __init__(self, cell_type,input_size, hidden_size, num_layers, drop, device, output_size, attention,num_of_heads, skip=False):
        
        super(RNN, self).__init__()
        cells = {
          "LSTM"    : nn.LSTM,
          "GRU"     : nn.GRU
        }
        
        self.cell_type = cell_type
        self.hidden_size = hidden_size
        self.attention = attention
        self.num_layers=num_layers
        self.skip= skip
        self.dropout = nn.Dropout(drop)
        
        if skip:
            self.rnn = nn.ModuleList()
            self.rnn.append(cells[cell_type](input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True, bidirectional=True, device=device))
            for i in range(num_layers-1):
                self.rnn.append(cells[cell_type](input_size=hidden_size*2, hidden_size=hidden_size, num_layers=1, batch_first=True,  bidirectional=True, device=device))
        else:
            self.rnn = cells[cell_type](input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True, dropout=drop, bidirectional=True, device=device)
        
        if self.attention:
            self.Q = nn.Linear(hidden_size*2,hidden_size*2, bias=False, device=device)
            self.K = nn.Linear(hidden_size*2,hidden_size*2, bias=False, device=device)
            self.V = nn.Linear(hidden_size*2,hidden_size*2, bias=False, device=device)
            self.att = nn.MultiheadAttention(self.hidden_size*2, num_heads=num_of_heads ,batch_first=True, device=device)

        self.out = nn.Linear(2*hidden_size, output_size, device=device)

    def forward(self, x, lengths):
#         print(x.data.shape)
#         x,_ = unpackedOutput= torch.nn.utils.rnn.pad_packed_sequence(x, batch_first=True)
#         print(x.shape)
        if self.skip:
            x_packed = torch.nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
            x_packed = x_packed.float().to(device)
            if self.cell_type == 'LSTM':
                r_out_packed, (h_n, h_c) = self.rnn[0](x_packed) 
                
            else: 
                r_out_packed, h_n = self.rnn[0](x_packed)
            r_out,_ = torch.nn.utils.rnn.pad_packed_sequence(r_out_packed, batch_first=True)
            for i in range(self.num_layers-1):
                prev = r_out
                r_out = self.dropout(r_out)
                x_packed = torch.nn.utils.rnn.pack_padded_sequence(r_out, lengths, batch_first=True, enforce_sorted=False)
                x_packed = x_packed.float().to(device)
                if self.cell_type == 'LSTM':
                    r_out_packed, (h_n, h_c) = self.rnn[i+1](x_packed) 
                else:
                    r_out_packed, h_n = self.rnn[i+1](x_packed)
                r_out,_ = torch.nn.utils.rnn.pad_packed_sequence(r_out_packed, batch_first=True)
                r_out = r_out + prev
        else:
            x_packed = torch.nn.utils.rnn.pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)
            x_packed = x_packed.float().to(device)
            if self.cell_type == 'LSTM':
                r_out_packed, (h_n, h_c) = self.rnn(x_packed) 
            else:
                r_out_packed, h_n = self.rnn(x_packed)
            r_out ,_ = torch.nn.utils.rnn.pad_packed_sequence(r_out_packed, batch_first=True)
        if self.attention:
            Q = self.Q(r_out)
            K = self.K(r_out)
            V = self.V(r_out)
            att, _ = self.att(Q, K, V)
            out = att + r_out 
        else:
            out = r_out
        out = self.out(torch.cat((out[:, -1, :hidden_size], out[:, 0, hidden_size:]), dim=1))
        probabilities = F.softmax(out, dim=1)
        return probabilities

# Dataset class

In [ ]:
# Define the dataset class
class TextDataset:
    def __init__(self, features, targets, lengths):
        self.features = features
        self.targets = targets
        self.lengths = lengths
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, item):
        return {
            "x": self.features[item],
            "y": self.targets[item],
            "l": self.lengths[item]
        }

# Initialize

In [ ]:
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader

hidden_size = 30
output_size = 3
num_layers = 2
cell_type = 'LSTM'
num_of_heads = 5
net = RNN(cell_type,  X_train.shape[2], hidden_size,num_layers,0.3, device, 3,False,num_of_heads, False)
net.to('cuda') 
EPOCH = 13
LR = 0.0012

optimizer = torch.optim.Adam(net.parameters(), lr=LR)   # optimize all net parameters
loss_func = nn.CrossEntropyLoss()

dataset = TextDataset(X_train, Y_train,X_train_length)
train_loader = DataLoader(dataset, batch_size=128, shuffle=True)

validation_set = TextDataset(X_valid, Y_valid, X_valid_length)
valid_loader = DataLoader(validation_set, batch_size=128, shuffle=False)

# Training

In [ ]:
# Start train mode
net.train()

# Accumulate losses
train_loss = []
valid_loss = []
# Count the number of batches given to the model
step_count = 0


# Iterate over the whole dataset for one epoch
for epoch in range(EPOCH):
    temp_train_loss = []
    temp_valid_loss = []
    # Generate batch data
    for step,data in enumerate(train_loader):
        b_x = data['x']
        b_y = data['y']
        b_l = data['l']
        
        # clear gradients for this training step
        optimizer.zero_grad()

        output = net(b_x,b_l)
        loss = loss_func(output, b_y)
        
        # For visualization        
        temp_train_loss.append(loss.item())
        step_count += 1

        # backpropagation - compute gradients
        loss.backward()
        nn.utils.clip_grad_norm_(net.parameters(), 1.0)
        # apply gradients
        optimizer.step()
    total_correct = 0
    total_samples = 0    
#         Do not calculate gradients
    with torch.no_grad():
        # Faster inference
        net.eval()
        for step, data in enumerate(valid_loader):
            b__x = data['x']
            b__y = data['y']
            b__l = data['l']
            
#             packed_inputs_valid = torch.nn.utils.rnn.pack_padded_sequence(b__x, b__l, batch_first=True, enforce_sorted=False)
#             packed_inputs_valid = packed_inputs_valid.float()
#             packed_inputs_valid = packed_inputs_valid.to('cuda')
            test_output = net(b__x,b__l)
            loss = loss_func(test_output, b__y)
            
            # For visualization        
            temp_valid_loss.append(loss.item())

            # Get predicted class
            pred_y = torch.argmax(test_output, dim=1).squeeze()
            
            # Calculate accuracy
            total_correct += (pred_y == b__y).sum().item()
            total_samples += b__y.size(0)
            
        net.train()
        
    accuracy = total_correct / total_samples
    train_l = mean(temp_train_loss)
    valid_l = mean(temp_valid_loss)
    train_loss.append(train_l)
    valid_loss.append(valid_l)
    # Return to training mode
    print('Epoch: ', epoch, '| train loss: %.4f' % train_l,'valid loss: %.4f' %valid_l , '| test accuracy: %.2f' % accuracy)


# Plot results
plt.figure(figsize=(12, 5))
plt.plot(train_loss, label='Train Loss')
plt.plot(valid_loss, label='Validation Loss')
plt.title('Learning Curve')
plt.xlabel('Epochs', fontsize=12)
plt.ylabel('Cross Entropy Loss', fontsize=12)
plt.legend()
plt.show()

In [ ]:
print(f'Make: {len(X_valid)} predictions')
print(len(X_valid))
test_output = net(X_valid,X_valid_length)

y_pred = torch.argmax(test_output.cpu(), dim=1).data.numpy().squeeze()

y_true = Y_valid.cpu().data.numpy().squeeze()

print(y_true, 'real number')
print(y_pred, 'prediction number')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

# Assuming you already have y_true and y_pred defined
# classification_report_str is a string returned by classification_report
classification_report_str = classification_report(y_true, y_pred)

# Parsing the classification report string
classification_report_lines = classification_report_str.split('\n')

# Extracting precision, recall, and F1 score from the second last line (class average line) of classification report
metrics = classification_report_lines[-2].split()
# Index 1 corresponds to precision, index 2 to recall, index 3 to f1-score
accuracy = accuracy_score(y_true, y_pred)
precision = metrics[2]
recall = metrics[3]
f1_score = metrics[4]

# Printing the precision, recall, and F1 score
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

# Printing the confusion matrix
print('Confusion Matrix:')
print(confusion_matrix(y_true, y_pred))
print(len(y_true))
plot_roc_curve(y_true, test_output)


# Create output file

In [ ]:
import csv

final_output = net(X_test,X_test_lenth)
predictions = torch.argmax(final_output.cpu(), dim=1).data.numpy().squeeze()
number_to_sentiment = {
    0: "NEGATIVE",
    1: "NEUTRAL",
    2: "POSITIVE",
}

array = [number_to_sentiment[value] for value in predictions]

with open("submission.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id", "Predicted"])

    for idx, word in enumerate(array, start=1):
        writer.writerow([idx, word])

# Optuna optimizer

In [ ]:
%pip install optuna

In [ ]:
import optuna

# 1. Define an objective function to be maximized.
def objective(trial):

    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_size =  trial.suggest_categorical('hidden_size', [2,10,30,64])
    output_size = 3
    cell_type = trial.suggest_categorical('cell_type', ['LSTM', 'GRU'])
    attention = trial.suggest_categorical('attention', [True, False])
    skip_connections = trial.suggest_categorical('skip_connections', [True, False])
    dropout = trial.suggest_categorical('dropout', [0.0,0.1,0.2,0.3,0.4])
    LR = trial.suggest_float('LR', 0.0001,0.1)
    gc = trial.suggest_categorical('gc', [True, False])
    num_of_heads = trial.suggest_categorical('num_of_heads' ,[1,2,3,4,5])
    net = RNN(cell_type,  X_train.shape[2], hidden_size,num_layers,dropout, device, output_size,attention,2,skip_connections)
    net.to('cuda') 
    print(net)
    EPOCH = trial.suggest_int('EPOCH',10,20)
    optimizer = torch.optim.Adam(net.parameters(), lr=LR)   # optimize all net parameters
    loss_func = nn.CrossEntropyLoss()
    
    net.train()

    # Accumulate losses
    train_loss = []
    valid_loss = []
    # Count the number of batches given to the model
    step_count = 0
    # Iterate over the whole dataset for one epoch
    for epoch in range(EPOCH):
        temp_train_loss = []
        temp_valid_loss = []
        # Generate batch data
        for step,data in enumerate(train_loader):
            b_x = data['x']
            b_y = data['y']
            b_l = data['l']

            # clear gradients for this training step
            optimizer.zero_grad()

            output = net(b_x,b_l)
            loss = loss_func(output, b_y)

            # For visualization        
            temp_train_loss.append(loss.item())
            step_count += 1

            # backpropagation - compute gradients
            loss.backward()
            if gc:
                nn.utils.clip_grad_norm_(net.parameters(), 1.0)
            # apply gradients
            optimizer.step()
        total_correct = 0
        total_samples = 0    
    #         Do not calculate gradients
        with torch.no_grad():
            # Faster inference
            net.eval()
            for step, data in enumerate(valid_loader):
                b__x = data['x']
                b__y = data['y']
                b__l = data['l']

    #             packed_inputs_valid = torch.nn.utils.rnn.pack_padded_sequence(b__x, b__l, batch_first=True, enforce_sorted=False)
    #             packed_inputs_valid = packed_inputs_valid.float()
    #             packed_inputs_valid = packed_inputs_valid.to('cuda')
                test_output = net(b__x,b__l)
                loss = loss_func(test_output, b__y)

                # For visualization        
                temp_valid_loss.append(loss.item())

                # Get predicted class
                pred_y = torch.argmax(test_output, dim=1).squeeze()

                # Calculate accuracy
                total_correct += (pred_y == b__y).sum().item()
                total_samples += b__y.size(0)

            net.train()

        accuracy = total_correct / total_samples
        train_l = mean(temp_train_loss)
        valid_l = mean(temp_valid_loss)
        train_loss.append(train_l)
        valid_loss.append(valid_l)
        # Return to training mode
        print('Epoch: ', epoch, '| train loss: %.4f' % train_l,'valid loss: %.4f' %valid_l , '| test accuracy: %.4f' % accuracy)


    # Plot results
    plt.figure(figsize=(12, 5))
    plt.plot(train_loss, label='Train Loss')
    plt.plot(valid_loss, label='Validation Loss')
    plt.title('Learning Curve')
    plt.xlabel('Epochs', fontsize=12)
    plt.ylabel('Cross Entropy Loss', fontsize=12)
    plt.legend()
    plt.show()
    
    print(f'Make: {len(X_valid)} predictions')
    print(len(X_valid))
    test_output = net(X_valid,X_valid_length)

    y_pred = torch.argmax(test_output.cpu(), dim=1).data.numpy().squeeze()

    y_true = Y_valid.cpu().data.numpy().squeeze()

    print(y_true, 'real number')
    print(y_pred, 'prediction number')
    
    classification_report_str = classification_report(y_true, y_pred)

    # Parsing the classification report string
    classification_report_lines = classification_report_str.split('\n')

    # Extracting precision, recall, and F1 score from the second last line (class average line) of classification report
    metrics = classification_report_lines[-2].split()
    # Index 1 corresponds to precision, index 2 to recall, index 3 to f1-score
    accuracy = accuracy_score(y_true, y_pred)
    precision = metrics[2]
    recall = metrics[3]
    f1_score = metrics[4]

    # Printing the precision, recall, and F1 score
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1_score}')

    # Printing the confusion matrix
    print('Confusion Matrix:')
    print(confusion_matrix(y_true, y_pred))
    print(len(y_true))
    plot_roc_curve(y_true, test_output)
    return float(f1_score)

# Run

I had make a run of 100 trials to find the optimal parameters, now for faster submission I write hyparameters hardcore and I change num of trials to 1

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)
print("Number of finished trials: ", len(study.trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_parallel_coordinate(study)